# 用Python实现的密码相关算法

1. 替换密码
    1. 简单替换密码
    1. 凯撒密码
    1. 替换密码的破解方法
    1. 利用频率分析破解
1. 防篡改
    1. 摘要算法如何防止篡改
    1. md5
    1. sha1
1. 非对称加密
    1. 加密与解密方法
    1. 数字签名

## 替换密码

### 凯撒密码

使用Python简单实现凯撒密码算法
输入：明文字符串
输出：密文字符串

In [3]:
def caesar_encrypt(plain_text: str) -> str:
    """
    凯撒密码加密函数
    :param plain_text: 待加密的明文（支持大小写字母、数字、标点等）
    :return: 加密后的密文
    """
    cipher_text = []
    shift = 3  # 凯撒密码的经典位移量为3
    # 遍历明文中的每个字符
    for char in plain_text:
        # 处理大写字母（A-Z，ASCII码：65-90）
        if 'A' <= char <= 'Z':
            # 计算偏移后的ASCII码：先归一化到0-25，偏移后取模26（防止超出字母范围），再还原为大写字母ASCII
            encrypted_ascii = (ord(char) - ord('A') + shift) % 26 + ord('A')
            cipher_text.append(chr(encrypted_ascii))
        # 处理小写字母（a-z，ASCII码：97-122）
        elif 'a' <= char <= 'z':
            # 同理，归一化到0-25，偏移取模后还原为小写字母ASCII
            encrypted_ascii = (ord(char) - ord('a') + shift) % 26 + ord('a')
            cipher_text.append(chr(encrypted_ascii))
        # 非字母字符（数字、标点、空格等）直接保留
        else:
            cipher_text.append(char)
    # 拼接字符列表为最终密文
    return ''.join(cipher_text)

In [4]:
# 示例用法
plain = "Hello, World! 123"
cipher = caesar_encrypt(plain)
print(f"明文: {plain}")
print(f"密文: {cipher}")

明文: Hello, World! 123
密文: Khoor, Zruog! 123


### ROT13
1. 仅对字母进行替换 （如 A→N、B→O、M→Z）从N开始（如 N→A、O→B、Z→M）
1. 对数字、标点符号、空格等非字母不做处理
1. 加密解密算法一致，对于密文执行本算法也可用于解密
1. 安全性极低，容易破解
1. 为凯撒密码的变种（凯撒密码位移量一般为3）

ROT47
和ROT13算法基本相同，不做详细介绍。

In [ ]:
$ echo "Hello" | tr 'A-Za-z' 'N-ZA-Mn-za-m' # 使用tr命令模拟rot13加密

`Uryyb` # rot13加密后的结果

In [ ]:
$ echo "Uryyb" | tr 'A-Za-z' 'N-ZA-Mn-za-m' # 解密rot13加密的字符串

`Hello` # rot13解密后的结果

使用Python实现ROT13加密解密算法
输入：明文字符串
输出：密文字符串

In [ ]:
def rot13_encrypt(plain_text: str) -> str:
    """
    手动实现ROT13加密算法（加密和解密同一函数）
    :param plain_text: 待加密的明文
    :return: ROT13加密后的密文
    """
    cipher_text = []
    for char in plain_text:
        # 处理大写字母（A-Z，ASCII码65-90）
        if 'A' <= char <= 'Z':
            # 步骤：归一化到0-25 → 偏移13位 → 取模26（循环偏移） → 还原为大写字母ASCII
            encrypted_ascii = (ord(char) - ord('A') + 13) % 26 + ord('A')
            cipher_text.append(chr(encrypted_ascii))
        # 处理小写字母（a-z，ASCII码97-122）
        elif 'a' <= char <= 'z':
            # 同上逻辑，针对小写字母处理
            encrypted_ascii = (ord(char) - ord('a') + 13) % 26 + ord('a')
            cipher_text.append(chr(encrypted_ascii))
        # 非字母字符（数字、标点、空格、中文等）直接保留
        else:
            cipher_text.append(char)
    # 拼接字符列表为最终密文
    return ''.join(cipher_text)

In [ ]:
# 示例用法
plain_text = "Hello, World! 123"
cipher_text = rot13_encrypt(plain_text)
print("明文:", plain_text)
print("密文:", cipher_text)

明文: Hello, World! 123
密文: Uryyb, Jbeyq! 123


In [ ]:
# Python提供的codesc模块中已经内置了rot13编解码器，可以直接使用
import codecs

def rot13_encrypt_with_codecs(plain_text: str) -> str:
    """
    基于Python内置codecs库实现ROT13加密（加密和解密同一函数）
    :param plain_text: 待加密的明文（支持大小写字母，非字母字符保留不变）
    :return: ROT13加密后的密文
    """
    # 核心：codecs.encode直接支持rot_13编码，加密解密同一操作
    return codecs.encode(plain_text, "rot_13")

In [ ]:
# 示例用法
plain_text = "Hello, World! 123"
cipher_text = rot13_encrypt_with_codecs(plain_text)
print("明文:", plain_text)
print("密文:", cipher_text)

明文: Hello, World! 123
密文: Uryyb, Jbeyq! 123


In [ ]:
# 解密方法与加密相同
cipher_text = "Uryyb, Jbeyq! 123" # 刚刚用ROT13加密得到的密文
decrypted_text = rot13_encrypt_with_codecs(cipher_text)
print("解密后明文:", decrypted_text)

解密后明文: Hello, World! 123


### 可自定义位移的凯撒密码加密算法

1. 凯撒密码位移通常为3
1. ROT13位移为13
1. ROT47位移为47

这些替换密码的加密算法区别就是位移不同，如果可以自定义位移的话，就可以写出通用的凯撒密码加密算法。

In [4]:
def caesar_encrypt(plain_text: str, shift: int = 3) -> str:
    """
    凯撒密码加密函数
    :param plain_text: 待加密的明文（支持大小写字母、数字、标点等）
    :param shift: 位移量（整数，可正可负；正数=向后偏移，负数=向前偏移）
    :return: 加密后的密文
    """
    cipher_text = []
    # 遍历明文中的每个字符
    for char in plain_text:
        # 处理大写字母（A-Z，ASCII码：65-90）
        if 'A' <= char <= 'Z':
            # 计算偏移后的ASCII码：先归一化到0-25，偏移后取模26（防止超出字母范围），再还原为大写字母ASCII
            encrypted_ascii = (ord(char) - ord('A') + shift) % 26 + ord('A')
            cipher_text.append(chr(encrypted_ascii))
        # 处理小写字母（a-z，ASCII码：97-122）
        elif 'a' <= char <= 'z':
            # 同理，归一化到0-25，偏移取模后还原为小写字母ASCII
            encrypted_ascii = (ord(char) - ord('a') + shift) % 26 + ord('a')
            cipher_text.append(chr(encrypted_ascii))
        # 非字母字符（数字、标点、空格等）直接保留
        else:
            cipher_text.append(char)
    # 拼接字符列表为最终密文
    return ''.join(cipher_text)

In [12]:
# 加密示例
plain = "Hello, World! 123"
cipher = caesar_encrypt(plain, shift=3)
print(f"明文: {plain}")
print(f"密文: {cipher}")

# 解密示例（位移量取负值）
decrypted = caesar_encrypt(cipher, shift=-3)
print(f"解密后明文: {decrypted}")

明文: Hello, World! 123
密文: Khoor, Zruog! 123
解密后明文: Hello, World! 123


### 替换密码的破解方法

替换密码往往明文和密文有一一对应的关系，利用这种关系我们知道比如密文中出现了一个字母A，那么所有密文中出现的A都对应明文中的同一个字母。通过穷举方法，我们可以用算出所有A可能对应的字母，再对这些输出进行人工识别，看能否组成有意义以的单词或短语，来进行破解。

In [17]:
def brute_force_caesar(encrypted_text: str) -> None:
    """
    穷举法破解凯撒密码（枚举所有26种位移量）
    :param encrypted_text: 凯撒密码加密后的密文
    :return: 无返回值，直接打印所有候选明文及对应位移量
    """
    print("=== 凯撒密码穷举法破解开始 ===")
    print(f"待解密密文：{encrypted_text}\n")
    # 枚举所有可能的位移量（0-25，共26种）
    for possible_shift in range(26):
        # 调用解密函数，获取对应位移量的明文
        candidate_plain = caesar_encrypt(encrypted_text, -possible_shift)
        # 打印位移量和候选明文，便于识别
        print(f"位移量 {possible_shift:2d} → 候选明文：{candidate_plain}")
    print("\n=== 穷举法破解结束：请根据语义识别有效明文 ===")

In [18]:
# 测试暴力破解凯撒加密
cipher = "Khoor, Zruog! 123"
brute_force_caesar(cipher)

=== 凯撒密码穷举法破解开始 ===
待解密密文：Khoor, Zruog! 123

位移量  0 → 候选明文：Khoor, Zruog! 123
位移量  1 → 候选明文：Jgnnq, Yqtnf! 123
位移量  2 → 候选明文：Ifmmp, Xpsme! 123
位移量  3 → 候选明文：Hello, World! 123
位移量  4 → 候选明文：Gdkkn, Vnqkc! 123
位移量  5 → 候选明文：Fcjjm, Umpjb! 123
位移量  6 → 候选明文：Ebiil, Tloia! 123
位移量  7 → 候选明文：Dahhk, Sknhz! 123
位移量  8 → 候选明文：Czggj, Rjmgy! 123
位移量  9 → 候选明文：Byffi, Qilfx! 123
位移量 10 → 候选明文：Axeeh, Phkew! 123
位移量 11 → 候选明文：Zwddg, Ogjdv! 123
位移量 12 → 候选明文：Yvccf, Nficu! 123
位移量 13 → 候选明文：Xubbe, Mehbt! 123
位移量 14 → 候选明文：Wtaad, Ldgas! 123
位移量 15 → 候选明文：Vszzc, Kcfzr! 123
位移量 16 → 候选明文：Uryyb, Jbeyq! 123
位移量 17 → 候选明文：Tqxxa, Iadxp! 123
位移量 18 → 候选明文：Spwwz, Hzcwo! 123
位移量 19 → 候选明文：Rovvy, Gybvn! 123
位移量 20 → 候选明文：Qnuux, Fxaum! 123
位移量 21 → 候选明文：Pmttw, Ewztl! 123
位移量 22 → 候选明文：Olssv, Dvysk! 123
位移量 23 → 候选明文：Nkrru, Cuxrj! 123
位移量 24 → 候选明文：Mjqqt, Btwqi! 123
位移量 25 → 候选明文：Lipps, Asvph! 123

=== 穷举法破解结束：请根据语义识别有效明文 ===


我们看到输出中有这样一行
`位移量  3 → 候选明文：Hello, World! 123`
这一行的输出可以识别出来是有意义的明文，其他输出都是杂乱的，所以我们可以确定原加密算法的位移量是3。

当然我们也可以根据一些英文常用词让计算机去掉一些无意义的破解结果，降低人工识别的工作量。

In [6]:
# 自动识别有效明文（基于英文常用单词库，提升便捷性）
def brute_force_caesar_auto(encrypted_text: str, common_words: list = None) -> list:
    """
    穷举法破解凯撒密码（自动筛选含常用英文单词的候选明文，减少人工判断）
    :param encrypted_text: 待解密密文
    :param common_words: 英文常用单词列表（默认提供基础常用词）
    :return: 筛选后的有效候选（包含位移量和明文）
    """
    # 默认英文常用单词（可根据需求扩展）
    if common_words is None:
        common_words = ['the', 'and', 'is', 'are', 'I', 'you', 'he', 'she', 'it', 'we', 'they', 'in', 'on', 'at', 'to']
    
    valid_candidates = []
    print("=== 凯撒密码自动穷举破解开始 ===")
    print(f"待解密密文：{encrypted_text}\n")

    for possible_shift in range(26):
        candidate_plain = caesar_encrypt(encrypted_text, -possible_shift)
        # 转换为小写，便于单词匹配（不区分大小写）
        candidate_lower = candidate_plain.lower()
        # 统计候选明文中包含的常用单词数量
        word_count = sum(1 for word in common_words if word.lower() in candidate_lower)
        # 若包含至少1个常用单词，视为有效候选
        if word_count > 0:
            valid_candidates.append({
                'shift': possible_shift,
                'plain_text': candidate_plain,
                'matched_word_count': word_count
            })
            print(f"有效位移量 {possible_shift:2d} → 明文：{candidate_plain}（匹配常用词数量：{word_count}）")
        else:
            # 可选：隐藏无效候选，减少输出冗余
            # print(f"位移量 {possible_shift:2d} → 候选明文：{candidate_plain}（无效，无常用词匹配）")
            pass

    if not valid_candidates:
        print("未筛选到有效明文，请查看完整穷举结果！")
    else:
        print(f"\n共筛选出 {len(valid_candidates)} 个有效候选明文")
    print("=== 自动穷举破解结束 ===")
    return max(valid_candidates, key=lambda x: x['matched_word_count'])


In [7]:
# 示例用法
cipher_text = "Wklv lv d whvw phvvdjh ixq!" 
valid_candidates = brute_force_caesar_auto(cipher_text)
if len(valid_candidates) > 0:
    print("筛选出的有效候选明文：")
    print(valid_candidates)

=== 凯撒密码自动穷举破解开始 ===
待解密密文：Wklv lv d whvw phvvdjh ixq!

有效位移量  0 → 明文：Wklv lv d whvw phvvdjh ixq!（匹配常用词数量：1）
有效位移量  1 → 明文：Vjku ku c vguv oguucig hwp!（匹配常用词数量：1）
有效位移量  2 → 明文：Uijt jt b uftu nfttbhf gvo!（匹配常用词数量：1）
有效位移量  3 → 明文：This is a test message fun!（匹配常用词数量：2）
有效位移量  7 → 明文：Pdeo eo w paop iaoowca bqj!（匹配常用词数量：1）
有效位移量  8 → 明文：Ocdn dn v ozno hznnvbz api!（匹配常用词数量：1）
有效位移量 13 → 明文：Jxyi yi q juij cuiiqwu vkd!（匹配常用词数量：1）
有效位移量 14 → 明文：Iwxh xh p ithi bthhpvt ujc!（匹配常用词数量：2）
有效位移量 15 → 明文：Hvwg wg o hsgh asggous tib!（匹配常用词数量：1）
有效位移量 21 → 明文：Bpqa qa i bmab umaaiom ncv!（匹配常用词数量：1）
有效位移量 23 → 明文：Znoy oy g zkyz skyygmk lat!（匹配常用词数量：1）
有效位移量 25 → 明文：Xlmw mw e xiwx qiwweki jyr!（匹配常用词数量：2）

共筛选出 12 个有效候选明文
=== 自动穷举破解结束 ===
筛选出的有效候选明文：
{'shift': 3, 'plain_text': 'This is a test message fun!', 'matched_word_count': 2}


In [ ]:
# 我们给出一段用替换密码加密的文本，要求读者尝试破解。
cipher_text = "Wklv lv pb vhfuhw phvvdjh."

### 验证身份证号是否有效

In [9]:
def verify_id_card_check_digit(id_card: str) -> bool:
    """
    验证18位身份证号的校验位（最后一位）是否合法
    :param id_card: 18位居民身份证号码（字符串格式，最后一位可为'X'/'x'）
    :return: 校验通过返回True，不通过返回False（含格式不合法的情况）
    """
    # 步骤1：先校验身份证号的基础格式（长度、字符合法性）
    if len(id_card) != 18:
        print("错误：身份证号长度不是18位！")
        return False
    
    # 前17位必须是数字，最后一位可以是数字或X/x
    for i in range(17):
        if not id_card[i].isdigit():
            print(f"错误：第{i+1}位（前17位）不是数字！")
            return False
    last_char = id_card[-1]
    if not (last_char.isdigit() or last_char.upper() == 'X'):
        print("错误：最后一位校验位不是数字或X/x！")
        return False
    
    # 步骤2：定义加权因子（第1位到第17位对应的权重）
    weight_factors = [7, 9, 10, 5, 8, 4, 2, 1, 6, 3, 7, 9, 10, 5, 8, 4, 2]
    
    # 步骤3：定义校验码对应表（余数0-10对应最终校验码）
    check_code_map = {0: '1', 1: '0', 2: 'X', 3: '9', 4: '8', 
                      5: '7', 6: '6', 7: '5', 8: '4', 9: '3', 10: '2'}
    
    # 步骤4：计算前17位数字与加权因子的乘积和
    total = 0
    for i in range(17):
        # 提取前17位的每一位数字（转为整数）
        digit = int(id_card[i])
        # 累加 数字 * 对应加权因子
        total += digit * weight_factors[i]
    
    # 步骤5：计算余数（对11取模）
    remainder = total % 11
    
    # 步骤6：获取预期校验码，并与实际最后一位比对
    expected_check_code = check_code_map[remainder]
    actual_check_code = id_card[-1].upper()  # 统一转为大写X，兼容小写x
    
    # 步骤7：返回比对结果
    is_valid = (actual_check_code == expected_check_code)
    if not is_valid:
        print(f"校验失败：预期校验位为{expected_check_code}，实际为{actual_check_code}")
    return is_valid

In [ ]:
# 示例用法

# 测试用例1：合法身份证号（示例，非真实有效身份信息，仅用于校验）
valid_id = "110101199003077336"
# 测试用例2：非法身份证号（最后一位校验位错误）
invalid_id = "110101199003077331"
# 测试用例3：格式错误（长度不足18位）
wrong_length_id = "11010119900307733"
# 测试用例4：格式错误（前17位含非数字）
wrong_char_id = "1101011990030773aX"

# 验证各用例
print(f"用例1（合法）校验结果：{verify_id_card_check_digit(valid_id)}")
print("-" * 30)
print(f"用例2（校验位错误）校验结果：{verify_id_card_check_digit(invalid_id)}")
print("-" * 30)
print(f"用例3（长度错误）校验结果：{verify_id_card_check_digit(wrong_length_id)}")
print("-" * 30)
print(f"用例4（字符错误）校验结果：{verify_id_card_check_digit(wrong_char_id)}")

校验失败：预期校验位为6，实际为X
用例1（合法）校验结果：False
------------------------------
校验失败：预期校验位为6，实际为1
用例2（校验位错误）校验结果：False
------------------------------
错误：身份证号长度不是18位！
用例3（长度错误）校验结果：False
------------------------------
错误：第17位（前17位）不是数字！
用例4（字符错误）校验结果：False
